<a href="https://colab.research.google.com/github/Saksham9804/Projects/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Imports

Importing all the libraries I need for building, training, visualizing, and saving my neural network.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import pandas as pd
import pickle

Checking if a GPU is available and using it if possible—otherwise, I’ll use the CPU.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Preparation

Defining a transform to turn images into tensors and normalize them so the model can train easily.

In [ ]:
image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

Now,loading the EMNIST Balanced dataset—both the training and test splits.

In [ ]:
train_set = datasets.EMNIST(root='./data', split='balanced', train=True, download=True, transform=image_transform)
test_set = datasets.EMNIST(root='./data', split='balanced', train=False, download=True, transform=image_transform)


Splitting the training data into a training set (80%) and a validation set (20%) so I can check if my model is overfitting.


In [ ]:
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = random_split(train_set, [train_size, val_size])

Creating data loaders to efficiently feed data in batches to my model.


In [ ]:
batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Model Definition

Here I’m defining a custom neural network for the EMNIST task.

In [ ]:
class EMNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        # I’m stacking convolutional layers with ReLU activations and adding batch normalization for stable learning.
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        # I’m flattening the output so it can go through fully connected layers next.
        self.flatten = nn.Flatten()
        # I’m adding a skip connection to help gradients flow and prevent vanishing.
        self.skip = nn.Linear(128 * 7 * 7, 256)
        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 47)  # 47 possible classes in EMNIST Balanced

    def forward(self, x):
        # I’m passing data through convolutions and flattening.
        x = self.features(x)
        x = self.flatten(x)
        # I’m creating a skip connection from feature output directly to fully connected output.
        skip_out = self.skip(x)
        x = torch.relu(self.fc1(x))
        x = x + skip_out  # Adding the skip connection for improved learning
        x = self.fc2(x)
        return x

Now, creating the model and sending it to the right device (CPU or GPU) and picking cross-entropy as my loss since this is a classification problem and Adam as my optimizer.


In [ ]:
model = EMNISTModel().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training Loop

In [ ]:
epochs = 10
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

Now looping over my specified number of epochs, training and validating each time.

In [ ]:
for epoch in range(epochs):
    # Training phase: I’m letting the model learn from the training data.
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(correct / total)
    # Validation phase: I’m checking the model’s performance on unseen validation data.
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(val_correct / val_total)

    # Printing progress after each epoch so I can see how my model is doing.
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.4f}")
    print(f"  Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.4f}")

# Plotting Results

Here, plotting the loss and accuracy curves so I can visualize training progress.

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Saving Model and Predictions

Saving the trained model’s weights to disk using pickle.

In [ ]:
with open('emnist_model.pkl', 'wb') as f:
    pickle.dump(model.state_dict(), f)
print("Model weights saved to emnist_model.pkl")


Switching to evaluation mode and predicting class labels for the test set.

In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

Now, saving the test predictions to a CSV file for submission or later use.

In [ ]:
submission = pd.DataFrame({'id': range(len(predictions)), 'label': predictions})
submission.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")